In [ ]:
import trimesh
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
import math
import itertools
import scipy
from networkx import Graph
from scipy.spatial import cKDTree
from numpy import linalg as LA
from scipy.linalg import expm, sinm, cosm
from shapely.geometry import Polygon, Point, MultiPoint
import plotly.graph_objs as go
import json

cube=trimesh.load_mesh('D:/DDP/ucube.stl')
sphere=trimesh.load_mesh('D:/DDP/usphere.stl')
sphere2=trimesh.load_mesh('D:/DDP/usphere2.stl')
shape=trimesh.load_mesh('D:/DDP/ushape.stl')
mug=trimesh.load_mesh('D:/DDP/mug.stl')
hsphere=trimesh.load_mesh('D:/DDP/h_sphere.stl')

mesh = cube
bounds=mesh.bounds
mini=bounds[0][2]
maxi=bounds[1][2]
length=bounds[1][0]-bounds[0][0]
breadth=bounds[1][1]-bounds[0][1]
area=length*breadth
height=maxi-mini

print("Bounding dimensions:")
print("Length: ", length)
print("Breadth: ", breadth)
print("Height: ", height)
print("Area: ", area)

mesh.show()

In [ ]:
def sampling(Slice_2D):
    M, N = math.ceil(length), math.ceil(breadth)
    density_factor = 1
    
    x = np.linspace((-M/2),(M/2),20*density_factor)
    y = np.linspace((-N/2),(N/2),20*density_factor)
    X,Y = np.meshgrid(x,y)
    
    polygon1 = Slice_2D.polygons_full[0]
    try:
        polygon2 = Slice_2D.polygons_full[1]
    except:
        pass
    
    
    grid_coord=[]
    for i in range(len(X[0])):
        for j in range(len(Y[0])):
            grid_coord.append((X[i][j],Y[i][j]))
            
    new_grid_coord=[]        
    for i in grid_coord:
        if polygon1.contains(Point(i))==1:
            new_grid_coord.append(i)
            
        try:
            if polygon2.contains(Point(i))==1:
                new_grid_coord.append(i)
        except:
            pass
        
    return new_grid_coord

layers=int(input("Number of layers: "))
layer_list=[mini]
c=mini
vert_layer={}
vert_list=[]
for i in range(layers-1):
    c += height/(layers-1)
    layer_list.append(c)
layer_list[0]+=0.01
layer_list[-1]-=0.01
co=0
for i in layer_list:
    slice_ = mesh.section(plane_origin=(0,0,i), plane_normal=[0,0,1])
    slice_2D, to_3D = slice_.to_planar()
    layer_ver =  np.asanyarray(sampling(slice_2D))
    vert_list.append(layer_ver)
    vert_layer[co] = layer_ver
    x,y = layer_ver.T
    plt.title("layer no. %i" %(co+1))
    co+=1
    plt.scatter(x,y,s=1)
    plt.xlim([bounds[0][0]-5, bounds[1][0]+5])
    plt.ylim([bounds[0][1]-5, bounds[1][1]+5])
    plt.show()

In [ ]:
l=range(layers+1)
tot=0
k=0
layer_vert_numbers={}
layer_nodes=[]
node_list=[]
for i in l[0:-1]:
    k = len(vert_list[i])
    layer_vert_numbers[i] = list(range(tot,tot+k))
    tot+=len(vert_list[i])
    node_list.append(k)
    print("layer-{}:".format(i+1), k, "nodes")
print("\nTotal nodes: ",tot)
print("\nMax nodes in a layer: ",max(node_list))

for i in range(layers):
    layer_nodes.append(layer_vert_numbers[i])

In [ ]:
def generate_edges(coor, euclid_dist):
    points=[]
    for i in range(len(coor)):
        points.append(tuple(coor[i]))
        
    tree = cKDTree(points)
    G = Graph()
    G.add_nodes_from(points)
    G.add_edges_from((point, points[idx2])
                     for idx1, point in enumerate(points)
                     for idx2 in tree.query_ball_point(point, euclid_dist)
                     if idx1 != idx2)
    return list(G.edges)

def conn(edj):
    c=[]
    for i in range(len(edj)):
        x=list(vertices2.values()).index(edj[i][0])
        y=list(vertices2.values()).index(edj[i][1])
        c.append([x,y])
    return c

def layer_conn(edj, points, layer_no):
    dict_points={}
    if layer_no == 0:
        all_layer_vert_numbers = layer_vert_numbers[layer_no]
    else:    
        all_layer_vert_numbers = layer_vert_numbers[layer_no-1]+layer_vert_numbers[layer_no]
    for i in range(len(points)):
        j = all_layer_vert_numbers[i]
        dict_points[j] = points[i]
        
    c=[]
    for i in range(len(edj)):
        x=list(dict_points.values()).index(edj[i][0])
        y=list(dict_points.values()).index(edj[i][1])
        c.append([list(dict_points.keys())[x],list(dict_points.keys())[y]])
    return c

def graphin3d(vertices, edges):
    x_coords = [v[0] for v in vertices.values()]
    y_coords = [v[1] for v in vertices.values()]
    z_coords = [v[2] for v in vertices.values()]
    vertices_scatter = go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=1, color="blue"), name='Vertices')
    
    x_coords_edges = []
    y_coords_edges = []
    z_coords_edges = []
    for edge in edges:
        x_coords_edges.append(vertices[edge[0]][0])
        x_coords_edges.append(vertices[edge[1]][0])
        x_coords_edges.append(None)
        y_coords_edges.append(vertices[edge[0]][1])
        y_coords_edges.append(vertices[edge[1]][1])
        y_coords_edges.append(None)
        z_coords_edges.append(vertices[edge[0]][2])
        z_coords_edges.append(vertices[edge[1]][2])
        z_coords_edges.append(None)
    edges_scatter = go.Scatter3d(x=x_coords_edges, y=y_coords_edges, z=z_coords_edges, mode='lines', line=dict(width=0.5, color='red'), name='Edges')

    layout = go.Layout(scene=dict(xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Z')))
    fig = go.Figure(data=[vertices_scatter, edges_scatter], layout=layout)
    fig.show()

merged = list(itertools.chain.from_iterable(vert_list))
all_vert=[]
coor2d={}

for i in merged:
    all_vert.append(i.tolist())
    
for i in range(len(all_vert)):
    coor2d[i]=all_vert[i]


coor1=[]
coor2=[]
vert_layer_3d1={}
vert_layer_3d2={}

for i in range(len(layer_list)):
    temp_coord1=[]
    temp_coord2=[]
    for j in range(len(vert_layer[i])):
        coor1.append(list(np.append(vert_layer[i][j], layer_list[i])))
        coor2.append(tuple(np.append(vert_layer[i][j], layer_list[i])))
        temp_coord1.append(list(np.append(vert_layer[i][j], layer_list[i])))
        temp_coord2.append(tuple(np.append(vert_layer[i][j], layer_list[i])))
    vert_layer_3d1[i]=temp_coord1
    vert_layer_3d2[i]=temp_coord2
vertices1={}
vertices2={}
for i in range(len(coor1)):
    vertices1[i]=coor1[i]
    vertices2[i]=coor2[i]

euclid_dist=input("Euclid distance: ")
#euclid_dist = math.sqrt((math.ceil(length)/(20*density_factor))**2 + (math.ceil(breadth)/(20*density_factor))**2)

print("No. of vertices:",len(coor2))
all_edges = conn(generate_edges(coor2, euclid_dist))
print("No. of edges:",len(all_edges))
print("\nTop View of the graph:")

import networkx as nx
import matplotlib.pylab as plt

G = nx.Graph()
G.add_nodes_from(vertices2)
G.add_edges_from(all_edges)
nx.draw(G, coor2d, node_size=10, with_labels=0)
plt.show()

graphin3d(vertices2, all_edges)

In [5]:
def adjacency_matrix(graph):
    adj = [[0 for node in graph[0]] for node in graph[0]]
    for edge in graph[1]:
        node1, node2 = edge[0], edge[1]
        adj[node1][node2] += 1
        adj[node2][node1] += 1
    return adj

def degree_matrix(Adj):
    deg_mat = np.diag(np.sum(np.array(Adj), axis=1)) 
    return deg_mat

def active_adj_mat(adj_mat, layer_no):
    if layer_no == 0:
        points= vert_layer_3d2[0]
        layer_conn(generate_edges(points,euclid_dist), points, layer_no)
        Adj = adjacency_matrix([range(len(adj_mat[0])), layer_conn(generate_edges(points,euclid_dist), points, layer_no)])
        return Adj
        
    else:
        points= vert_layer_3d2[layer_no-1] + vert_layer_3d2[layer_no]
        layer_conn(generate_edges(points,euclid_dist), points, layer_no)
        Adj = adjacency_matrix([range(len(adj_mat[0])),layer_conn(generate_edges(points,euclid_dist), points, layer_no)])
        return Adj

def active_deg_mat(active_adj_mat):
    act_deg_mat = np.diag(np.sum(np.array(active_adj_mat), axis=1)) 
    return act_deg_mat

def get_laplacian(diagonal_mat, adj_mat):
    laplcaian_mat = np.subtract(np.array(diagonal_mat), np.array(adj_mat))
    return laplcaian_mat

def get_eigen_val(laplacian_mat):
    eigen_val, eigen_vec = LA.eig(laplacian_mat)
    return eigen_val

def get_eigen_vec(laplacian_mat):
    eigen_val, eigen_vec = LA.eig(laplacian_mat)
    return eigen_vec

def get_eigen_vector(laplacian_mat):
    eigen_val, eigen_vec = LA.eig(laplacian_mat)
    return eigen_val, eigen_vec

def get_zero_rows(laplacian):
    index_list = []
    for i in range(len(laplacian)):
        if all(j == 0 for j in laplacian[i]):
            index_list.append(i)
    return index_list

def trunc(values, decs):
    return np.trunc(values*10**decs)/(10**decs)

In [ ]:
A = adjacency_matrix([range(len(coor1)),all_edges])
df = pd.DataFrame(A)
print("Adjacency Matrix:")
print(df)

D = degree_matrix(A)
df = pd.DataFrame(D)
print("\nDegree Matrix:")
print(df)

L = get_laplacian(D,A)
df = pd.DataFrame(L)
print("\nLaplacian Matrix:")
print(df)

EVec = get_eigen_vec(L)
#EVec = trunc(EVec, 6)
df = pd.DataFrame(EVec)
print("\nEigenvector Matrix:")
print(df)

In [ ]:
ambient_temp = 298
t_init = [ambient_temp for i in range(len(vertices2))]
t_history = {0:t_init}

def heat_nodes(layer, metal_fusion_temp,t_initial):
    t_initial_c = t_initial.copy()
    nodes = active_nodes(vertices2,vert_layer_3d2,layer)
    for node in nodes:
        t_initial_c[node] = metal_fusion_temp
    return t_initial_c

def active_nodes(vertices,vert_layer_3d,layer_no):
    active_nodes = []
    for i in vert_layer_3d[layer_no]:
        for key, values in vertices.items():
            if values == i:
                active_nodes.append(key)
    return active_nodes 

def apply_heat(eigen_vec, eigen_val, t_heat):
    eigen_vec_inv = np.linalg.inv(eigen_vec)
    eigen_val_mat = np.diag(eigen_val)
    matrix_exp = expm(-1 * 3000000 *0.0000034 *0.021* eigen_val_mat)
    t_final_heat = np.matmul(np.matmul(np.matmul(eigen_vec, matrix_exp), eigen_vec_inv), t_heat)
    return t_final_heat

def apply_diffusion(adj,t_heated, layer):
    a = active_adj_mat(A, layer)
    d = active_deg_mat(a)
    l = get_laplacian(d, a)
    eigen_val, eigen_vec = get_eigen_vector(l)
    t_final_diffusion = apply_heat(eigen_vec, eigen_val, t_heated)

    return t_final_diffusion

def apply_convection(layer,temp):
    t_convection = temp.copy()
    act_nodes = active_nodes(vertices2,vert_layer_3d2,layer)
    t_conv = convection(act_nodes, t_convection)
    return t_conv

def convection(acti_nodes,tempp):
    coefficient = 0.01
    temperature_convection = tempp.copy()
    for node in range(len(tempp)):
        if node in acti_nodes:
            temperature_convection[node] = math.exp(-1 * 0.021 * coefficient) * (tempp[node] - 298) + 298

    return temperature_convection

def real_t_history(layer_no):
    return np.array([c.real for c in t_history[layer_no]])

'''def temp_labels(t_history):
    temp_arr = []
    for layer in range(layers):
        temp_arr.append(t_history[layer][temp_node])
    return temp_arr, np.mean(temp_arr), np.std(temp_arr)'''

In [ ]:
t_post_cooling_list=[]

for layer in range(layers):
    t_heated1 = heat_nodes(layer,1260,t_init)  # layer by layer
    t_post_diff = apply_diffusion(A,t_heated1,layer)
    t_post_cooling = apply_convection(layer,t_post_diff)
    #print(t_post_cooling)
    t_init = t_post_cooling.copy()
    for r in range(len(t_init)):
        if t_init[r] > 1260:
            t_init[r] = t_init[r-1]
        elif t_init[r] < 0 :
            t_init[r] = 1259.79800121
            
    print(t_init)
    
    t_history[layer + 1] = t_init
    
    
    #temp_array, temp_mean, temp_std = self.get_temp_labels(temperature_history, discretised_part, flat_dict)
        #return temperature_history, temp_array, temp_mean, temp_std'''

In [ ]:
df = pd.DataFrame(t_history)
df_new1=df
del df_new1[0]
#df_new1=df_new.rename(columns={0:"layer-0",1:"layer-1"})
df_new1

In [ ]:
df_new1.plot(y=1, kind = 'line')
df_new1.plot(y=2, kind = 'line')
df_new1.plot(y=3, kind = 'line')
df_new1.plot(y=4, kind = 'line')
df_new1.plot(y=5, kind = 'line')
df_new1.plot(y=6, kind = 'line')
df_new1.plot(y=7, kind = 'line')
df_new1.plot(y=8, kind = 'line')
df_new1.plot(y=9, kind = 'line')
df_new1.plot(y=10, kind = 'line')
plt.show()

In [ ]:
def heatgraphin3d(vertices, edges, temperatures):
    x_coords = [v[0] for v in vertices.values()]
    y_coords = [v[1] for v in vertices.values()]
    z_coords = [v[2] for v in vertices.values()]
    vertices_scatter = go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=5, color=temperatures,colorscale='Jet', showscale=True), name= 'Vertices')

    x_coords_edges = []
    y_coords_edges = []
    z_coords_edges = []
    for edge in edges:
        x_coords_edges.append(vertices[edge[0]][0])
        x_coords_edges.append(vertices[edge[1]][0])
        x_coords_edges.append(None)
        
        y_coords_edges.append(vertices[edge[0]][1])
        y_coords_edges.append(vertices[edge[1]][1])
        y_coords_edges.append(None)
        
        z_coords_edges.append(vertices[edge[0]][2])
        z_coords_edges.append(vertices[edge[1]][2])
        z_coords_edges.append(None)
    edges_scatter = go.Scatter3d(x=x_coords_edges, y=y_coords_edges, z=z_coords_edges, mode='lines', line=dict(width=0, color='black'), name='Edges')
    
    
    data = [vertices_scatter, edges_scatter]
    layout = go.Layout(scene=dict(xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Z')))
    fig = go.Figure(data=data, layout=layout)
    fig.show()  

for layer in range(1, layers+1):
    print("layer-", layer)
    heatgraphin3d(vertices2, all_edges, real_t_history(layer))

In [ ]:
# Thermal stress matrix: ε_th = α * (T - T_0)
# α, Thermal expansion coeff = 13.9 x 10^-6 /K
t_init = [ambient_temp for i in range(len(vertices2))]
T_stress = 13.9 * 10**-6 * (real_t_history(layers) - t_init)

df = pd.DataFrame(T_stress)
print("\nThermal stress matrix:")
print(df)

# Elastic strain matrix: ε_el = S * (ε_th - ε_pl)
# S, Elastic modulus = 204 GPa
# ε_pl is assumed to be 0

E_Strain = 204 * 10**9 * T_stress
df = pd.DataFrame(E_Strain)
print("\nElastic strain matrix:")
print(df)

# Residual stress matrix: σ_res = E * (ε_el - ε_0)
# E, Yield strength = 460 MPa
# ε_0 is assumed to be 0

Res_Stress = 460 * 10**6 * E_Strain
df = pd.DataFrame(Res_Stress)
print("\nResidual stress matrix:")
print(df)